## Read data

In [17]:
import json
import pandas as pd

with open('../data/occurrences_8f3a026f-2fc4-419c-9918-f0684a2abbec.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

df = pd.DataFrame(data['data'])

### Shots by months

In [18]:
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%dT%H:%M:%S.%fZ')
df['month'] = df['date'].dt.to_period('M')
df['year'] = df['date'].dt.year
df['time'] = df['date'].dt.time

df.groupby(['month']).agg(
    count=('id', 'count'),
    min_date=('date', 'min'),
    max_date=('date', 'max'),
).reset_index()

,month,count,min_date,max_date
0,2025-01,50,2025-01-01,2025-01-15 14:00:00
